Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [2]:
train.head()

id                                        description  ratingCategory
0  1321  \nSometimes, when whisky is batched, a few lef...               1
1  3861  \nAn uncommon exclusive bottling of a 6 year o...               0
2   655  \nThis release is a port version of Amrut’s In...               1
3   555  \nThis 41 year old single cask was aged in a s...               1
4  1965  \nQuite herbal on the nose, with aromas of dri...               1

In [3]:
test.head()

id                                        description
0  3461  \nStyle: Speyside single malt scotch Color: Wa...
1  2604  \nVery bright and lively, with a nice balance ...
2  3341  \nA new oloroso-forward Chivas positioned to s...
3  3764  \nAged in bourbon casks and then enhanced in R...
4  2306  \nThere is a freshness to the wood on the nose...

In [5]:
# Splitting Data Components

train_data = train['description']
target = train['ratingCategory']

### Define Pipeline Components

In [6]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Defining Vectorizer
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

# Defining Classifier
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [7]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20),
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train_data, target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  2.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [8]:
# Calculating the Best Accuracy Score achieved by the pipeline
grid_search.best_score_

0.7247395654086601

In [9]:
# Best hyperparameters discovered by the pipeline
grid_search.best_params_

{'clf__max_depth': 20,
 'vect__max_df': 1.0,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

## Using Randomized Search

In [10]:
import scipy
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'vect__max_df': scipy.stats.uniform(0.70, 0.99),
    'vect__min_df': scipy.stats.uniform(.02, .05),
    'vect__max_features': scipy.stats.randint(500,1000),
    'clf__n_estimators': scipy.stats.randint(10, 50),
    'clf__max_depth': scipy.stats.randint(15,20)
}

rand_search = RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=-1, n_iter=25)
rand_search.fit(train_data, target)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   57.5s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [11]:
# Calculating Score
rand_search.best_score_

0.7225387771469955

In [12]:
# Calculating best parameters
rand_search.best_params_

{'clf__max_depth': 18,
 'clf__n_estimators': 41,
 'vect__max_df': 1.6004396508803285,
 'vect__max_features': 534,
 'vect__min_df': 0.05760733862326976}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [19]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [20]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [13]:
subNumber = 0

In [21]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

### Randomized Search Submission File

In [ ]:
# Predictions on test sample
pred = rand_search.predict(test['description'])

submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
submission.to_csv(f'./data/submission_rand{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [14]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                  algorithm='randomized',
                  n_iter=10)

In [21]:
lsi = Pipeline([('vect', vect), ('svd', svd)])
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [18]:
# params = {'lsi__svd_n_components': scipy.stats.randint(10,100),
#           'lsi__vect_max_df': scipy.stats.uniform(0.8, 0.99),
#           'clf__n_estimators': scipy.stats.randint(10,100)
    
# }

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [26]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(train_data, target)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  7.8min
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed: 25.6min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [34]:
# Best Score
grid_search.best_score_

0.7242508671177574

In [33]:
# Best Parameters
grid_search.best_params_

{'clf__max_depth': 15,
 'lsi__svd__n_components': 100,
 'lsi__vect__max_df': 0.75}

### Using Randomized Search

In [37]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'lsi__svd__n_components': scipy.stats.randint(10,250),
    'lsi__vect__max_df': scipy.stats.uniform(0.75, 1.0),
    'clf__max_depth': scipy.stats.randint(5,20)
}

rand_search = RandomizedSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
rand_search.fit(train_data, target)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 14.6min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed: 15.8min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [38]:
# Calculating score
rand_search.best_score_

0.7222921835207226

In [39]:
# Best Params
rand_search.best_params_

{'clf__max_depth': 19,
 'lsi__svd__n_components': 108,
 'lsi__vect__max_df': 0.8559604629703795}

### Make a Submission File

In [40]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [41]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [42]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [43]:
subNumber = 1

In [44]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [45]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

In [47]:
# Continue Word Embedding Work Here
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")

In [48]:
# Function for extracting vectors using spacy

def get_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [69]:
# Vector Matrix for train data
X = get_vectors(train['description'])

In [70]:
# Vector Matrix for test data
X_test = get_vectors(test['description'])

In [51]:
len(X) == len(train['description'])

True

In [61]:
X[0]

array([-4.67550606e-02,  2.17486411e-01, -1.39248788e-01, -8.63811150e-02,
        1.08062603e-01,  8.21812302e-02, -6.99262992e-02, -7.88967833e-02,
       -4.63048331e-02,  1.67534256e+00, -8.29739422e-02,  8.47098455e-02,
       -1.61168594e-02, -1.15673296e-01, -1.00931458e-01, -1.48674801e-01,
       -2.86327489e-02,  1.00717258e+00, -8.84611458e-02, -3.00116055e-02,
       -2.12408844e-02, -1.98691525e-03, -2.18204875e-02, -2.43686680e-02,
       -8.50067884e-02, -5.28534129e-02, -3.88755389e-02,  2.85331737e-02,
        9.71467644e-02, -1.52220830e-01, -7.67232990e-03,  3.68110649e-02,
        3.03680412e-02, -3.20375822e-02,  4.51759845e-02, -2.22320929e-02,
        2.99556088e-02, -3.53264883e-02, -8.09046626e-02,  4.02572937e-02,
       -4.52380776e-02,  4.00653407e-02,  1.01217575e-01, -1.02363355e-01,
        6.43703416e-02,  1.21800080e-01, -8.15031528e-02, -7.51190484e-02,
        4.85178418e-02,  3.61067131e-02, -5.62809184e-02,  4.18828800e-02,
       -9.56438202e-03, -

In [54]:
train.columns

Index(['id', 'description', 'ratingCategory'], dtype='object')

In [52]:
# Instantiating Classifier
rfc = RandomForestClassifier()

In [55]:
# Fitting data

rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [73]:
rfc.score(X, train['ratingCategory'])

1.0

### Make a Submission File

In [72]:
# Predictions on test sample
pred = rfc.predict(X_test)

In [74]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [75]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [76]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?